### Estimación de S_xx a partir de FDD

In [ ]:
peak_selec = 1 # nro de pico a estimar
peak_width = 30 # ancho de banda a estimar (en indices)
idxes = [mode_ind_fdd[peak_selec]- peak_width//2, mode_ind_fdd[peak_selec] + peak_width//2]
f_S_hat = f_S[idxes[0]:idxes[1]]
n_modes = 2 # cantidad de modos considerados

In [ ]:
H_shape = (S_xx.shape[1], S_xx.shape[1]*len(f_S_hat))
H = np.zeros(H_shape)
H = np.reshape(S_xx[idxes[0]:idxes[1], :, :], H_shape, 'F')

w_S = 2*np.pi*f_S
w_S_hat = 2*np.pi*f_S_hat

D = np.zeros((len(f_S_hat), n_modes, n_modes), dtype=complex)
for mode in range(D.shape[1]):
    lamb = w_S[mode_ind_fdd[peak_selec]]*(-Xi[mode] + 1j*np.sqrt(1-Xi[mode]**2))
    D[:, mode, mode] = 1/(1j*w_S_hat - lamb)
    
modes_fdd_3d = np.repeat(modes_fdd.T[np.newaxis, :n_modes, :], len(w_S_hat), axis=0)
pre_M_fdd = np.matmul(D, modes_fdd_3d, dtype=complex)
dofs = pre_M_fdd.shape[2]
M_fdd = np.empty_like((pre_M_fdd.reshape((pre_M_fdd.shape[1], pre_M_fdd.shape[0]*dofs),order='C')))
i=0
while i < pre_M_fdd.shape[0]:
    j = i*dofs
    M_fdd[:,j:j+dofs] = pre_M_fdd[i,:,:]
    i += 1
    
tau = np.matmul(H,np.linalg.pinv(M_fdd))

S_xx_hat = np.repeat(tau[np.newaxis, :, :], len(w_S_hat), axis=0)@D@modes_fdd_3d

In [ ]:
plt.semilogy(f_S[:len(f_S)//4], abs(S_xx[:len(f_S)//4,0,0]))
plt.semilogy(f_S_hat, abs(S_xx_hat[:,0,0]))

In [ ]:
plt.semilogy(f_S_hat, abs(S_xx[idxes[0]:idxes[1],1,0]))
plt.semilogy(f_S_hat, abs(S_xx_hat[:,1,0]))